In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import gdal
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set max columns and rows displayed
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

In [3]:
# Read in csv files using pandas
accessolar_data = pd.read_csv("../data/ACCESSolar/NYCHA_ACCESSolar_Opportunities.csv")
utility_data = pd.read_csv("../data/Typical_Utility_Bill_Information_Electric__Beginning_2011.csv")

In [4]:
# Checking the ACCESSolar data to see if it came in right. Will need some cleaning but the read looks solid. 
accessolar_data.head()

,DEVELOPMENT,STREET ADDRESS,POSTCODE,BOROUGH,BLDG. NUMBER,TDS NUMBER,BIN,No. of FLOORS,ROOF SPACE (Sq. Ft.),ESTIMATED ROOF SOLAR CAPACITY (kW),ADJUSTED VALUE (kW),ROOF CONDITION RATING OR REPLACEMENT DATE,No. of RESIDENTIAL UNITS,No. of TOTAL UNITS,SENIOR DEVELOPMENT (YES/NO),Latitude,Longitude,Community Board,Council District,Census Tract,BBL,NTA,Location 1
0,POLO GROUNDS TOWERS,3005 FREDERICK DOUGLASS BOULEVARD,10039,MANHATTAN,8,149,1810098,2.0,19804.34,67.25,47.07,3,0.0,0,Non-residential,NaN,NaN,10,9,24302,1021060003,Central Harlem North-Polo Grounds,NaN
1,ROBBINS PLAZA,341 EAST 70TH STREET,10021,MANHATTAN,1,218,1044841,20.0,8774.21,29.32,29.32,5,150.0,150,Yes,40.766730,-73.957495,8,5,126,1014450023,Lenox Hill-Roosevelt Island ...,"(40.76673, -73.957495)"
2,ISAACS,1806 1ST AVENUE,10128,MANHATTAN,3,139,1082365,24.0,7942.03,33.06,33.06,3,206.0,207,No,40.781684,-73.945848,8,5,152,1015730001,Yorkville ...,"(40.781684, -73.945848)"
3,ADAMS,815 EAST 152ND STREET,10455,BRONX,7,118,2091989,21.0,6279.41,43.92,30.74,2,143.0,143,No,40.815451,-73.905192,1,17,79,2026650001,Melrose South-Mott Haven North ...,"(40.815451, -73.905192)"
4,HIGHBRIDGE GARDENS,1135 UNIVERSITY AVENUE,10452,BRONX,4,78,2095218,14.0,7112.23,40.67,28.47,2,117.0,118,No,40.837014,-73.928344,4,16,193,2025270032,Highbridge ...,"(40.837014, -73.928344)"


In [5]:
# Same check but on utility billing data. This looks bad - potentially a flawed convert from a formatted excel sheet?
# Going to check the source to see if the data is available in other formats. 
# UPDATE: seems like the source data is just... like this. Strange that a dataset this small would be this poorly formatted. 
# On the plus side, I traced the data back to the original source (NY Dept of Public Services) which has better-formatted and more recent data. 
utility_data.head()

,Effective Date,Season,Service Type,Company,Customer Type,Customer Details,Demand,Load Factor,Usage,Units of Usage,Total Bill
0,01-Jul-11,SUMMER 2011,ELECTRIC,0,RESIDENTIAL,-,-,-,0,KWH,22.45
1,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,250,0,50.45
2,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,500,0,78.45
3,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,750,0,106.45
4,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,1500,0,190.46


In [6]:
# Overwriting previous utility data with new data adapted from original data source
utility_data = pd.read_csv("../data/Typical Bill- Elecric-Residential- Winter 2013- 8-21-13.csv")

In [7]:
# Checking the read in for THIS version of the data
# Needs some cleaning, but the read looks good! 
utility_data.head()

,Company,Charge Type,0 kWh,250 kWh,500 kWh,750 kWh,1500 kWh,3000 kWh,5000 kWh
0,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,BASIC SERVICE CHARGE,$24.00,$24.00,$24.00,$24.00,$24.00,$24.00,$24.00
1,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,DELIVERY CHARGE,$ -,$12.41,$24.82,$37.22,$74.45,$148.89,$248.15
2,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,BILL CREDIT,$ -,$ -,$ -,$ -,$ -,$ -,$ -
3,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,TEMPORARY STATE ASSESSMENT SURCHGE,$ -,$0.83,$1.67,$2.50,$5.00,$9.99,$16.65
4,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,REVENUE DECOUPLING MECHANISM,$ -,$0.40,$0.81,$1.21,$2.42,$4.83,$8.05


In [8]:
# Checking if I can read in NSRDB shape files using geopandas. It works, both for the .dbf and .shp format.
# Unsure if we'll be using this - current plan is to interact directly with the PVWatts calculator - but it's good to know we can work with the files. 
geo_test = gpd.read_file("../data/NSRDB/nsrdb_v3_0_1_1998_2016_dni/nsrdb_v3_0_1_1998_2016_dni.shp")
geo_test.head()

,dni,gid,geometry
0,4.944,1,"POLYGON ((-56.76000 3.95410, -56.72000 3.95410..."
1,5.016,2,"POLYGON ((-56.76000 0.11410, -56.72000 0.11410..."
2,5.112,3,"POLYGON ((-56.48000 -15.36590, -56.44000 -15.3..."
3,5.040,4,"POLYGON ((-56.44000 4.87410, -56.40000 4.87410..."
4,4.296,5,"POLYGON ((-56.36000 -2.28590, -56.32000 -2.285..."


In [9]:
# Attempting to read in one of the rasters using GDAL
raster_test = gdal.Open("../data/Air_Pollution/AnnAvg1_10_300mRaster/aa1_bc300m/dblbnd.adf")

In [10]:
# It's a beautiful thing
type(raster_test)

osgeo.gdal.Dataset

In [11]:
# We can definitely access the rasters and their properties. I'm not sure exactly how to use them in conjunction with the data read into pandas
print("Driver: {}/{}".format(raster_test.GetDriver().ShortName,
                            raster_test.GetDriver().LongName))
print("Size is {} x {} x {}".format(raster_test.RasterXSize,
                                    raster_test.RasterYSize,
                                    raster_test.RasterCount))
print("Projection is {}".format(raster_test.GetProjection()))
geotransform = raster_test.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: AIG/Arc/Info Binary Grid
Size is 157 x 156 x 1
Projection is PROJCS["NAD83 / New York Long Island",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",41.03333333333333],PARAMETER["standard_parallel_2",40.66666666666666],PARAMETER["latitude_of_origin",40.16666666666666],PARAMETER["central_meridian",-74],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["Foot_US",0.3048006096012192],AXIS["X",EAST],AXIS["Y",NORTH]]
Origin = (912815.3055, 273282.0654)
Pixel Size = (984.0, -984.0)


In [12]:
# Let's see what we can get out of these binary files in the shadow data. 
# If the format is difficult to work with, it will probably be better to reverse-engineer the lookup script they included with the download. 
with open("../data/Shadows/slippy-nyc-dec-21/10/300/385.bin", mode='rb') as file:
    fileContent = file.read()

In [13]:
# Let's look at the content
# Commenting this out to save screen real estate, but hey, the data made it! The data and metadata are pretty bare-bones though, so while it would theoretically
# be possible to construct our own lookup based on the slippy hierarchy, it's definitely going to be MUCH more time-efficient to reverse-engineer theirs. 
#fileContent

In [14]:
# Okay, now the actual cleaning and EDA
accessolar_data.shape

(325, 23)

In [15]:
# Okay, wow, that's pretty small. We may need a larger dataset for this. off to get that now...

In [16]:
# Got building footprints shapefile, reading that in with Geopandas
building_geo = gpd.read_file("../data/Building_Footprints/geo_export_8d7509d4-c875-49c9-b71c-f79e6148cd9f.dbf")
building_geo.head()

,base_bbl,bin,cnstrct_yr,doitt_id,feat_code,geomsource,groundelev,heightroof,date_lstmo,time_lstmo,lststatype,mpluto_bbl,name,shape_area,shape_len,geometry
0,3044520815,3394646.0,2009.0,1212853.0,2100.0,Photogramm,18.0,21.608508,2017-08-22,00:00:00.000,Constructed,3044520815,None,854.662433,125.079796,"POLYGON ((-73.87130 40.65717, -73.87136 40.657..."
1,4030640041,4548330.0,1930.0,1226227.0,5110.0,Photogramm,122.0,10.360000,2017-08-17,00:00:00.000,Constructed,4030640041,None,217.594243,60.225858,"POLYGON ((-73.87671 40.71425, -73.87677 40.714..."
2,4139430001,4460479.0,1960.0,581946.0,2100.0,Photogramm,10.0,29.811570,2017-08-22,00:00:00.000,Constructed,4139430001,None,946.427476,123.141941,"POLYGON ((-73.85195 40.66235, -73.85195 40.662..."
3,3049720006,3355684.0,1920.0,858061.0,5110.0,Photogramm,32.0,11.200000,2017-08-17,00:00:00.000,Constructed,3049720006,None,248.678169,63.940817,"POLYGON ((-73.94029 40.64108, -73.94034 40.641..."
4,3055100055,3131737.0,1915.0,568078.0,2100.0,Photogramm,44.0,24.980000,2017-08-22,00:00:00.000,Constructed,3055100055,None,1163.227669,165.608763,"POLYGON ((-73.98999 40.62384, -73.98998 40.623..."


In [17]:
# There's a second version of the file in the directory, let's see what it is.
# Okay, this is a version with points instead of polygons - not what I want
building_geo_2 = gpd.read_file("../data/Building_Footprints/geo_export_b656bacd-288f-4b70-a9d7-ff8022520d88.dbf")
building_geo_2.head()

,base_bbl,bin,cnstrct_yr,doitt_id,feat_code,geomsource,groundelev,heightroof,date_lstmo,time_lstmo,lststatype,mpluto_bbl,name,geometry
0,3044520815,3394646.0,2009.0,1212853.0,2100.0,Photogramm,18.0,21.608508,2017-08-22,00:00:00.000,Constructed,3044520815,None,POINT (-73.87136 40.65721)
1,4030640041,4548330.0,1930.0,1226227.0,5110.0,Photogramm,122.0,10.360000,2017-08-17,00:00:00.000,Constructed,4030640041,None,POINT (-73.87674 40.71427)
2,4139430001,4460479.0,1960.0,581946.0,2100.0,Photogramm,10.0,29.811570,2017-08-22,00:00:00.000,Constructed,4139430001,None,POINT (-73.85201 40.66233)
3,3049720006,3355684.0,1920.0,858061.0,5110.0,Photogramm,32.0,11.200000,2017-08-17,00:00:00.000,Constructed,3049720006,None,POINT (-73.94032 40.64111)
4,3055100055,3131737.0,1915.0,568078.0,2100.0,Photogramm,44.0,24.980000,2017-08-22,00:00:00.000,Constructed,3055100055,None,POINT (-73.98990 40.62388)


In [18]:
# Converting geodataframe to a regular dataframe
building_data = pd.DataFrame(building_geo)

In [19]:
# Whole lot of buildings in NYC
building_data.shape

(1084824, 16)

In [20]:
# Kind of odd that the BBL fields are both objects, but shouldn't be an issue
building_data.dtypes

base_bbl        object
bin            float64
cnstrct_yr     float64
doitt_id       float64
feat_code      float64
geomsource      object
groundelev     float64
heightroof     float64
date_lstmo      object
time_lstmo      object
lststatype      object
mpluto_bbl      object
name            object
shape_area     float64
shape_len      float64
geometry      geometry
dtype: object

In [21]:
# Heavy skew apparent in shape_area, but that's to be expected. Some discrepancies in the counts for height, elevation, and area. 
building_data.describe()

,bin,cnstrct_yr,doitt_id,feat_code,groundelev,heightroof,shape_area,shape_len
count,1.084824e+06,1.073840e+06,1.084824e+06,1.084815e+06,1.084292e+06,1.082160e+06,1.084824e+06,1.084824e+06
mean,3.710452e+06,1.939137e+03,5.913684e+05,2.698394e+03,5.509678e+01,2.755853e+01,1.597378e+03,1.553366e+02
std,1.026705e+06,2.894405e+01,3.595879e+05,1.202137e+03,4.151002e+01,2.134018e+01,5.600513e+03,1.271791e+02
min,1.000002e+06,1.000000e+00,1.000000e+00,0.000000e+00,-1.400000e+01,0.000000e+00,3.247526e+01,2.373699e+01
25%,3.128746e+06,1.920000e+03,2.867168e+05,2.100000e+03,2.500000e+01,1.835000e+01,6.522663e+02,1.097161e+02
50%,4.066886e+06,1.930000e+03,5.736075e+05,2.100000e+03,4.600000e+01,2.608496e+01,9.573754e+02,1.379313e+02
75%,4.481876e+06,1.955000e+03,8.621152e+05,2.100000e+03,7.400000e+01,3.107057e+01,1.320558e+03,1.644648e+02
max,5.799523e+06,2.021000e+03,1.294010e+06,5.110000e+03,5.560000e+02,1.550000e+03,1.171874e+06,2.001729e+04


In [22]:
# Definitely have some nulls, but remarkably none at all in BIN or shape_area
building_data.isnull().sum()

base_bbl            3
bin                 0
cnstrct_yr      10984
doitt_id            0
feat_code           9
geomsource        216
groundelev        532
heightroof       2664
date_lstmo          0
time_lstmo          0
lststatype        192
mpluto_bbl        903
name          1082942
shape_area          0
shape_len           0
geometry            0
dtype: int64

In [23]:
# These three can probably be left off
building_data[building_data['base_bbl'].isnull()]

,base_bbl,bin,cnstrct_yr,doitt_id,feat_code,geomsource,groundelev,heightroof,date_lstmo,time_lstmo,lststatype,mpluto_bbl,name,shape_area,shape_len,geometry
107558,None,2129524.0,NaN,1292475.0,2100.0,Other (Man,183.0,54.0,2020-05-12,00:00:00.000,Constructed,None,RIVERDALE MONUMENT,204.865538,57.482674,"POLYGON ((-73.90835 40.88926, -73.90830 40.889..."
1056349,None,2044929.0,2020.0,1292634.0,5100.0,Other (Man,76.0,26.0,2020-06-25,00:00:00.000,Marked for Construction,None,None,824.998553,139.999934,"POLYGON ((-73.85086 40.84795, -73.85103 40.847..."
1057135,None,3425737.0,2020.0,1293505.0,2100.0,Other (Man,80.0,60.0,2020-11-20,00:00:00.000,Constructed,None,None,2982.695198,219.791677,"POLYGON ((-73.96207 40.68096, -73.96190 40.680..."


In [24]:
# None of the areas are zero, excellent
(building_data['shape_area'] == 0).sum()

0

In [25]:
# And a check for duplicates
building_data.duplicated().sum()

0

In [26]:
# Now the same routine with the other shapefile
address_geo = gpd.read_file("../data/Address_Points/geo_export_6a9aaf3b-b9d7-4c15-a922-0eb4efd2555d.dbf")

In [27]:
address_geo.head()

,address_id,bin,borocode,date_creat,time_creat,full_stree,h_no,hn_rng,hn_rng_suf,hno_suffix,hyphen_typ,date_modif,time_modif,physicalid,post_direc,post_modif,post_type,pre_direct,pre_modifi,pre_type,side_of_st,special_co,st_name,zipcode,geometry
0,1030290.0,1034253.0,1,2009-04-06,00:00:00.000,5 AVE,560,None,None,None,N,2015-09-25,00:00:00.000,1851.0,None,None,AVE,None,None,None,1,V,5,10036,POINT (-73.97941 40.75609)
1,3002257.0,3001479.0,3,2009-02-13,00:00:00.000,COLUMBIA HTS,25,None,None,None,N,2016-09-09,00:00:00.000,100296.0,None,None,HTS,None,None,None,1,V,COLUMBIA,11201,POINT (-73.99427 40.70194)
2,89223.0,4080420.0,4,2008-09-23,00:00:00.000,FLUSHING AVE,1903,None,None,None,N,2015-07-16,00:00:00.000,23102.0,None,None,AVE,None,None,None,1,V,FLUSHING,11385,POINT (-73.91888 40.71278)
3,1027498.0,1070766.0,1,2009-04-06,00:00:00.000,W END AVE,205,None,None,None,N,2016-09-09,00:00:00.000,4140.0,None,None,AVE,W,None,None,1,V,END,10023,POINT (-73.98638 40.77857)
4,5178668.0,2114036.0,2,2017-04-28,00:00:00.000,ARLINGTON AVE,3205,None,None,None,N,None,None,61034.0,None,None,AVE,None,None,None,2,V,ARLINGTON,10463,POINT (-73.91270 40.88418)


In [28]:
address_data = pd.DataFrame(address_geo)

In [29]:
# We definitely have fewer rows here than in building footprints
address_data.shape

(965131, 25)

In [30]:
# So many objects with three floats just... floating around in there. Won't be much for .describe to work with, but still worth a look. 
address_data.dtypes

address_id     float64
bin            float64
borocode        object
date_creat      object
time_creat      object
full_stree      object
h_no            object
hn_rng          object
hn_rng_suf      object
hno_suffix      object
hyphen_typ      object
date_modif      object
time_modif      object
physicalid     float64
post_direc      object
post_modif      object
post_type       object
pre_direct      object
pre_modifi      object
pre_type        object
side_of_st      object
special_co      object
st_name         object
zipcode         object
geometry      geometry
dtype: object

In [31]:
# Counts are at least showing as even! But the BIN minimum showing as zero is a little concerning
address_data.describe()

,address_id,bin,physicalid
count,9.651310e+05,9.651310e+05,965131.000000
mean,2.528769e+06,3.549309e+06,62717.260707
std,2.350241e+06,1.086066e+06,36732.020084
min,7.000000e+00,0.000000e+00,3.000000
25%,2.583385e+05,3.071522e+06,34109.000000
50%,3.006884e+06,4.006773e+06,60500.000000
75%,3.280040e+06,4.249084e+06,86621.000000
max,1.017986e+07,5.799523e+06,189873.000000


In [32]:
# Yeah, that ain't right. Only five, at least. 
(address_data['bin'] == 0).sum()

5

In [33]:
# These may have to go
address_data[address_data['bin'] == 0]

,address_id,bin,borocode,date_creat,time_creat,full_stree,h_no,hn_rng,hn_rng_suf,hno_suffix,hyphen_typ,date_modif,time_modif,physicalid,post_direc,post_modif,post_type,pre_direct,pre_modifi,pre_type,side_of_st,special_co,st_name,zipcode,geometry
958546,87967.0,0.0,4,2008-09-23,00:00:00.000,116 ST,83-67,None,None,None,Q,2021-01-20,00:00:00.000,19008.0,None,None,ST,None,None,None,1,None,116,11418,POINT (-73.83577 40.70466)
958548,9205251.0,0.0,4,2011-06-15,00:00:00.000,116 ST,83-71,None,None,None,Q,2021-01-20,00:00:00.000,19008.0,None,None,ST,None,None,None,1,None,116,11418,POINT (-73.83569 40.70453)
961213,5161024.0,0.0,1,2014-07-18,00:00:00.000,W 23 ST,512,None,None,None,N,2020-04-01,00:00:00.000,129100.0,None,None,ST,W,None,None,1,None,23,10011,POINT (-74.00485 40.74779)
962114,5166553.0,0.0,3,2015-07-28,00:00:00.000,N PORTLAND AVE,107,None,None,None,N,2020-06-16,00:00:00.000,182459.0,None,None,AVE,N,None,None,1,None,PORTLAND,11205,POINT (-73.97582 40.69456)
964228,5184455.0,0.0,1,2017-11-29,00:00:00.000,W 44 ST,511,None,None,None,N,2020-04-03,00:00:00.000,72759.0,None,None,ST,W,None,None,2,None,44,10036,POINT (-73.99520 40.76129)


In [34]:
# Nulls almost exclusively showing up in the optional fields, so that's good at least. Just the one null ZIP
address_data.isnull().sum()

address_id         0
bin                0
borocode           0
date_creat         0
time_creat         0
full_stree         0
h_no               0
hn_rng        946268
hn_rng_suf    964679
hno_suffix    942521
hyphen_typ         0
date_modif    715798
time_modif    715798
physicalid         0
post_direc    959322
post_modif    964274
post_type      22127
pre_direct    845237
pre_modifi    964981
pre_type      943255
side_of_st         0
special_co    951960
st_name            0
zipcode            1
geometry           0
dtype: int64

In [35]:
# How many rows in these two dataframes are for Brooklyn? That's our scope
(address_data['borocode'] == "3").sum()

308112

In [36]:
(building_data['base_bbl'].str.startswith("3")).sum()

331351

In [37]:
# And a check for duplicates
address_data.duplicated().sum()

0

In [38]:
# Trimming down to just Brooklyn
brooklyn_address_data = address_data.drop(address_data[address_data['borocode'] != "3"].index)

In [39]:
brooklyn_address_data.shape

(308112, 25)

In [40]:
# Removing those problem rows with BIN == 0 and dropping the address point geometry
brooklyn_address_data.drop(brooklyn_address_data[brooklyn_address_data['bin'] == 0].index, inplace=True)
brooklyn_address_data.drop(columns='geometry', inplace=True)

In [41]:
brooklyn_address_data.shape

(308111, 24)

In [42]:
# Lots of duplicate BINs in the address data
brooklyn_address_data.duplicated('bin').sum()

34251

In [43]:
# Only a few in the building data, but there shouldn't be any there
building_data.duplicated('bin').sum()

9

In [44]:
# Ah, okay, these are stand-in BINs where only the borocode is given and the actual building lacks an identifier.
building_data[building_data.duplicated('bin', False)]['bin'].value_counts()

4000000.0    6
5000000.0    3
3000000.0    3
Name: bin, dtype: int64

In [45]:
# Dropping the rows with stand-in BINs
building_data.drop(building_data[building_data.duplicated('bin', False)].index, inplace=True)

In [46]:
building_data.shape

(1084812, 16)

In [48]:
# Joining the address data for brooklyn with the relevant building data on BIN
brooklyn_data = brooklyn_address_data.merge(building_data, how='left', on='bin')

In [49]:
# Excellent
brooklyn_data.head()

,address_id,bin,borocode,date_creat,time_creat,full_stree,h_no,hn_rng,hn_rng_suf,hno_suffix,hyphen_typ,date_modif,time_modif,physicalid,post_direc,post_modif,post_type,pre_direct,pre_modifi,pre_type,side_of_st,special_co,st_name,zipcode,base_bbl,cnstrct_yr,doitt_id,feat_code,geomsource,groundelev,heightroof,date_lstmo,time_lstmo,lststatype,mpluto_bbl,name,shape_area,shape_len,geometry
0,3002257.0,3001479.0,3,2009-02-13,00:00:00.000,COLUMBIA HTS,25,None,None,None,N,2016-09-09,00:00:00.000,100296.0,None,None,HTS,None,None,None,1,V,COLUMBIA,11201,3002040001,1924.0,318201.0,2100.0,Photogramm,28.0,143.374389,2017-08-22,00:00:00.000,Constructed,3002040001,None,27247.461587,878.921970,"POLYGON ((-73.99442 40.70207, -73.99440 40.702..."
1,5125956.0,3332515.0,3,2011-08-09,00:00:00.000,1 AVE,5100,None,None,None,N,2015-10-01,00:00:00.000,42327.0,None,None,AVE,None,None,None,2,V,1,11232,3008030005,1931.0,82649.0,2100.0,Photogramm,17.0,28.780000,2017-08-22,00:00:00.000,Constructed,3008030005,None,81267.843494,1379.939037,"POLYGON ((-74.02089 40.65036, -74.02082 40.650..."
2,5126065.0,3246020.0,3,2011-08-12,00:00:00.000,W END AVE,177,None,None,A,N,2015-10-01,00:00:00.000,98122.0,None,None,AVE,W,None,None,1,V,END,11235,3087320114,1920.0,632731.0,2100.0,Photogramm,7.0,18.930000,2017-08-22,00:00:00.000,Constructed,3087320114,None,871.096641,128.835025,"POLYGON ((-73.95261 40.57757, -73.95267 40.577..."
3,5126067.0,3245988.0,3,2011-08-12,00:00:00.000,W END AVE,177,None,None,B,N,2015-10-01,00:00:00.000,98122.0,None,None,AVE,W,None,None,1,V,END,11235,3087320015,1920.0,755083.0,2100.0,Photogramm,7.0,18.700000,2017-08-22,00:00:00.000,Constructed,3087320015,None,769.542259,117.193081,"POLYGON ((-73.95251 40.57754, -73.95260 40.577..."
4,5136408.0,3247656.0,3,2012-08-27,00:00:00.000,EMMONS AVE,3047,None,None,A,N,2015-10-01,00:00:00.000,98154.0,None,None,AVE,None,None,None,2,V,EMMONS,11235,3087970065,1931.0,35118.0,2100.0,Photogramm,8.0,24.139143,2017-08-22,00:00:00.000,Constructed,3087970065,None,2264.424891,242.002301,"POLYGON ((-73.93697 40.58413, -73.93697 40.584..."


In [50]:
brooklyn_data.shape

(308111, 39)

In [51]:
# Looks like only 26 addresses didn't find a BIN to match up with, which is frankly way better than I expected
brooklyn_data.isnull().sum()

address_id         0
bin                0
borocode           0
date_creat         0
time_creat         0
full_stree         0
h_no               0
hn_rng        298545
hn_rng_suf    307902
hno_suffix    301670
hyphen_typ         0
date_modif    233785
time_modif    233785
physicalid         0
post_direc    307321
post_modif    307919
post_type      13593
pre_direct    243169
pre_modifi    308081
pre_type      295545
side_of_st         0
special_co    305778
st_name            0
zipcode            0
base_bbl          27
cnstrct_yr      1731
doitt_id          26
feat_code         27
geomsource        76
groundelev       121
heightroof       646
date_lstmo        26
time_lstmo        26
lststatype        77
mpluto_bbl       196
name          308039
shape_area        26
shape_len         26
geometry          26
dtype: int64

In [53]:
brooklyn_data.to_csv("../data/brooklyn_data.csv", index=False)